## European Medicine Agency (EMA)

The European Medicines Agency (EMA) is a decentralized agency of the European Union (EU) responsible for the scientific evaluation, supervision, and safety monitoring of medicines in the EU. EMA plays a crucial role in the public health domain by ensuring that all medicines available on the EU market are safe, effective, and of high quality.


<br/>

### Structure of an SmPC
The structure of the SmPC is defined by the European pharmaceutical legislation. The information included in the SmPC should be product specific and can be cross-referenced to avoid any redundancy. It should be documented in a clear language and should not lead to any ambiguity. The SmPC is divided into 6 major sections:

1. Name of the product
2. Composition
3. Pharmaceutical Form
4. Clinical particulars – Includes therapeutic indications, recommendation for dosages and safety information
5. Pharmacological properties – Takes into account the therapeutic indications of the clinical elements and their potential adverse drug reactions
6. Pharmaceutical particulars – Includes Regulatory information related to the drug

<br/>

<img src="./images/structureSmPC.png" width="600px;">

<br/>
<br/>

### Prescribing medicines for special populations
When prescribing medication to specific groups like children, the elderly, pregnant and breastfeeding women, and individuals with kidney or liver issues, it's
crucial to exercise caution. The public documentation for medicines, especially the Summary of Product Characteristics (SmPC) and the package leaflets (PLs), often includes many of these essential precautions.
<br/>

<img src="./images/SmPC_authorization_process.png" width="800px;">

When seeking marketing authorization, a company submits an application dossier that contains instructions for healthcare professionals on the safe and effective use of the medicine. In Europe, this is referred to as the Summary of Product Characteristics (SmPC). The SmPC must be regularly updated over the medicine's lifecycle to reflect new data on efficacy or safety. The figure above illustrates the connection between a medicine's development, the regulatory dossier with the initial proposed SmPC, the SmPC that gets approved, and subsequent updates to the SmPC.


[1]: https://www.ema.europa.eu/en/glossary/summary-product-characteristics
[2]: https://www.freyrsolutions.com/what-is-an-smpc


### Project Overview

Using natural language processing (NLP) techniquesto automatically extract adverse drug reactions from such unstructured textual information helps clinicalexperts to effectively and efficiently use them in daily practices. Such techniques have been developed for Structured Product Labels from the Food and Drug Administration (FDA), but there is no researchfocusing on extracting from the Summary of Product Characteristics.


In this work, we built a naturallanguage processing pipeline that automatically scrapes the summary of product characteristics online and then extracts adverse drug reactions from them.

<img src="./images/Project_Overview.png" width="300px;">

##  STEP 1: Download medicine data and import them to Database
You can download the European Medicines Agency's (EMA) medicine-related data published on this website in Excel table format from this page. EMA updates these medicine data tables once a day. The following two files need to be downloaded:

<br/>

* The file listing all medicines, **including the withdrawn and not authorised ones**, named medicines_output_european_public_assessment_reports_en.xlsx, found [here](https://www.ema.europa.eu/sites/default/files/Medicines_output_european_public_assessment_reports.xlsx)

and

* The file listing **all the URLs for the SmPCs**, named Product information URLs for member product-information-urls-member-states_en.xls, found [here](https://www.ema.europa.eu/documents/product-information/product-information-urls-member-states_en.xls)


##### Disclaimer
The database is used purely to join the two csv. The view can also be done with dataframes without the need to use the database

[3]: https://toolbox.eupati.eu/resources/prescribing-medicines-for-special-populations/?print=print
[4]: https://www.ema.europa.eu/en/homepage


In [1]:
import os
import pandas as pd
import re
import sqlite3
import camelot

# Define the folder that you store the required xls
PATH = "./data"

MEDICINE_LIST = "medicines_output_european_public_assessment_reports_en.xlsx"
PRODUCT_URLS_LIST = "product-information-urls-member-states_en.xls"

In [ ]:
medicinesOutputData = pd.read_excel(os.path.join(PATH, MEDICINE_LIST))
medicinesOutputData.head()

In [ ]:
productInformationURLs = pd.read_excel(os.path.join(PATH, PRODUCT_URLS_LIST))
productInformationURLs.head()

In [ ]:
conn = sqlite3.connect(os.path.join(PATH, "SmPCs.db"))

medicinesOutputData.to_sql('emaMedicinesOP', conn, if_exists='replace', index=False)
productInformationURLs.to_sql('emaProductInformationURLs', conn, if_exists='replace', index=False)

## Step 2: Create csv with SmPCs URLs

 The actual SmPCs information will be extracted from the joinedHumanAuthorisedProductURLsONLY.csv file which contains the URLs for the authorised medicines for human use.

In [ ]:
joinedHumanAuthorisedProductURLsONLY = pd.read_sql("""
    SELECT [URL(currentwebsite)]
    FROM emaMedicinesOP
    LEFT JOIN emaProductInformationURLs 
    ON emaMedicinesOP.[Medicine name]=emaProductInformationURLs.[ProductName]
    WHERE  emaMedicinesOP.Category='Human'
      AND emaMedicinesOP.[Authorisation Status] ='Authorised';
""", conn)

joinedHumanAuthorisedProductURLsONLY.to_csv(os.path.join(PATH, "joinedHumanAuthorisedProductURLsONLY.csv"))

## Step 3: Download SmPCs pdfs


In [ ]:
import requests

In [ ]:
urlsDF = pd.read_csv(os.path.join(PATH, "joinedHumanAuthorisedProductURLsONLY.csv"))
urlsDF.dropna(inplace = True)
urls = urlsDF['URL(currentwebsite)'].tolist()

In [ ]:
for index, url in enumerate(urls):
    response = requests.get(url)
    file_name = url.split('/')[-1]

    print("Download ", index, " with name: ", file_name)

    if response.status_code == 200:
        with open(os.path.join(PATH, "pdfs", file_name), 'wb') as pdf_file:
            pdf_file.write(response.content)
    else:
        print(f"Failed to download the : {file_name}")

### Step 4: Extract Tabulated Summary of Adverse Reactions from the 4.8 section 

This section should include all adverse reactions for which, after thorough assessment, a causal relationship between the medicinal product and the adverse event is at least a reasonable possibility. These ADRs should be all reactions from:
* Clinical trials
* Post-authorisation safety studies
* Spontaneous reporting

#####  The tabulated list of adverse reactions in section 4.8:
* Introduce table with short paragraph stating source of safety database
* Separate tables are acceptable in exceptional cases where adverse profiles markedly differ depending on the use of the product.

#####  Structure of the table: general considerations

* Table should be written according to MedDRA system organ classification (SOC)
* A pragmatic approach to the location of terms should be taken in order to make the identification of adverse reactions simpler and clinically appropriate for the reader
* Within each SOC, adverse reactions should be ranked under headings of frequency, most frequent reactions first
* Frequency grouping, adverse reactions should be presented in order of decreasing seriousness

##### Frequency Grouping:
* Very common (≥1/10)
* Common (≥1/100 to <1/10)
* Uncommon (≥1/1,000 to <1/100)
* Rare (≥1/10,000 to <1/1,000)
* Very rare (<1/10,000)
* Frequency not known (cannot be estimated from the available data)

#### SmPCs Examples

<img src="./images/SmPCs_example.png" width="500px;">


## MedDRA
MedDRA is a multilingual terminology allowing most users to operate in their native languages. The table below identifies the initial MedDRA version when each MedDRA language was made available to users. 

<br />

<img src="./images/MedDRA_Hierarchy.png" width="500px;">

<br />

The most important reason to “code” data into a standardised terminology is to analyse it. A key benefit of MedDRA is in its support of straightforward as well as sophisticated analyses. MedDRA can be used to analyse individual medical events (e.g., “Influenza”) or issues involving a system, organ or aetiology (e.g., infections) using its hierarchical structure. MedDRA can be used for signal detection and monitoring of clinical syndromes whose symptoms encompass numerous systems or organs using its multiaxial hierarchy or through the special feature of Standardised MedDRA Queries.

<br />

### MedDRA Hierarchy

The structure of MedDRA is very logical. There are five levels to the MedDRA hierarchy, arranged from very specific to very general.

<br />

Download [MedDRA codes](https://www.ema.europa.eu/en/documents/other/meddra-important-medical-event-terms-list-version-261_en.xlsx) and extract the unique values from the SOC Name column

In [60]:
medra = pd.read_excel(os.path.join(PATH, "MedDRA", "meddra-important-medical-event-terms-list-version-261_en.xlsx"))
socList = [".*" + term.replace(",", "").replace(" ", ".*") + ".*" for term in medra['SOC Name'].unique()]
SOCListRegex = '(?:% s)' % '|'.join(socList)
regexSOC = re.compile(SOCListRegex)

In [3]:
extra_terms = ["Infections – bacterial" , "Infections – viral", "Infections – pathogen unspecified",  "Infections – fungal"]

In [59]:
medra = pd.read_excel(os.path.join(PATH, "MedDRA", "meddra-important-medical-event-terms-list-version-261_en.xlsx"))
ptList = list(medra['PT Name'].unique()) # + extra_terms
PTListRegex = '(?:% s)' % '|'.join(ptList) 
regexPT = re.compile(PTListRegex)

In [57]:
pattern = re.compile("System\s*organ\s*class", re.IGNORECASE)
freq_regex = re.compile("(?i)(Very common|Common|Uncommon|Rare|Very rare|Frequency not known|Not known)")


for file in [
    # 3 Columns
        # Line-seperated
        'abecma-epar-product-information_en.pdf',
    
        # Comma-seperated
        # 'abseamed-epar-product-information_en.pdf',

        # 2 Columns
        #'abiraterone-accord-epar-product-information_en.pdf'
    
        # AR in the first row
        # Comma-seperated
        'abevmy-epar-product-information_en.pdf',
    
        # Line-seperated
        'abilify-maintena-epar-product-information_en.pdf',
    ]: 

    if file.lower().endswith('.pdf'):
        filename = file.split('/')[-1].split('-')[0]
        fullpath = os.path.join(PATH, 'pdfs', file)
        
        print("Start Processing Filename: ", fullpath)

        tables = camelot.read_pdf(fullpath, pages='all', flavor='lattice', line_scale=80, shift_text = ['', 'l'], process_background=False)

        for i, table in enumerate(tables):
            df = table.df
            AD_list = []
            FR_list = []
            SOC_list = []
            
            # Tables that have SOC in the first row 
            if df[0].str.contains(regexSOC, regex=True).any():
                print(f"Table {i} has SOC elements")
                print("The table: ", i, "1st Row: " , df.iloc[0])

        
                if len(df.iloc[0]) == 3:
                    print("Case A: Table has 3 columns SOC, AR, FR")
    
                    for index, row in df.iterrows():
                        if index == 0:
                            continue
                            
                        for item in row:
                            print("Item: ", item.replace("\n", ""))
                            if item:
                                if freq_regex.search(item):
                                    FR_list += item.split('\n')
                                elif regexSOC.search(item.replace("\n", "")):
                                    SOC_list += item.split('\n')
                                else:
                                    AD_list += item.split('\n')

                elif len(df.iloc[0]) == 2:
                    print("Case B: Table has 2 columns SOC, AR,FR")

                    for index, row in df.iterrows():
                            if index == 0:
                                continue
                                
                            for item in row:
                                print("Item: ", item)
                                if item:
                                    if freq_regex.search(item):
                                        phrases = item.split('\n')

                                        for phrase in phrases:
                                            freq, ar = phrase.split(':')
                                            print("x: ", x)
                                            print("freq: ", freq)
                                            print("ar: ", ar)
                                            

                elif df.iloc[0].astype(str).str.contains(freq_regex).any():
                    print("Case C: Table has frequencies in the first row")
                    
                    for index, row in df.iterrows():
                        if index == 0:
                            frequencies_dict = row.to_dict()
                            frequencies_dict.pop(0)
                            print("Frequencies: ", frequencies_dict)
                            continue

                        count = 0
                        for item in row:
                            if count == 0:
                                count += 1
                                continue

                            if item:
                                if ',' in item:
                                    ar_list = [item.replace("\n", "") for item in item.split(',') if len(item) > 2]
                                else:
                                    ar_list = [item.replace("\n", "") for item in re.findall(r'(?:[A-Z][a-z\s]+)', item) if len(item) > 2] 
                                    print(f"AR list: {ar_list}")
                                
                                AD_list += ar_list
                                FR_list += [frequencies_dict[count] for _ in range(len(ar_list))]

                            count += 1
                            
                pd.DataFrame({'ADR': AD_list, 'FREQ': FR_list}).to_csv(os.path.join(PATH, "extracted_tables", filename + ".csv"), sep=';', encoding='utf-8', index=False,  mode='a', header=False)

Start Processing Filename:  ./data/pdfs/abecma-epar-product-information_en.pdf


/var/folders/ks/61tstkzx4rj7v0my891xbvx40000gn/T/ipykernel_41646/4132718398.py:39: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if df[0].str.contains(regexSOC, regex=True).any():


Table 3 has SOC elements
The table:  3 1st Row:  0      System organ class
1        Adverse reaction
2    All grades frequency
Name: 0, dtype: object
Case A: Table has 3 columns SOC, AR, FR
Item:  Infections and infestationsa
Item:  Infections – bacterial Infections – viral Infections – pathogen unspecified Infections – fungal
Item:  Very common Very common  Very common Common
Item:  Blood and lymphatic system disorders
Item:  Neutropenia Leucopenia Thrombocytopenia Febrile neutropenia Lymphopenia Anaemia Disseminated intravascular coagulation
Item:  Very common Very common  Very common Very common  Very common Very common  Common
Item:  Immune system disorders
Item:  Cytokine release syndrome Hypogammaglobulinaemia Haemophagocytic lymphohistiocytosis*
Item:  Very common Very common Common
Item:  Metabolism and nutrition disorders
Item:  Hypophosphataemia Hypokalaemia Hyponatraemia Hypocalcaemia Hypoalbuminaemia Decreased appetite Hypomagnesaemia
Item:  Very common Very common  Very co

/var/folders/ks/61tstkzx4rj7v0my891xbvx40000gn/T/ipykernel_41646/4132718398.py:39: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if df[0].str.contains(regexSOC, regex=True).any():
/var/folders/ks/61tstkzx4rj7v0my891xbvx40000gn/T/ipykernel_41646/4132718398.py:81: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  elif df.iloc[0].astype(str).str.contains(freq_regex).any():
/var/folders/ks/61tstkzx4rj7v0my891xbvx40000gn/T/ipykernel_41646/4132718398.py:39: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if df[0].str.contains(regexSOC, regex=True).any():
/var/folders/ks/61tstkzx4rj7v0my891xbvx40000gn/T/ipykernel_41646/4132718398.py:81: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually g

Table 0 has SOC elements
The table:  0 1st Row:  0    System organ \nclass
1             Very common
2                  Common
3                    Rare
4               Very rare
5               Not known
Name: 0, dtype: object
Case C: Table has frequencies in the first row
Frequencies:  {1: 'Very common', 2: 'Common', 3: 'Rare', 4: 'Very rare', 5: 'Not known'}
AR list: ['Necrotising fasciitisa']
AR list: ['Anaphylactic shock']
AR list: ['Dehydration']
AR list: ['Hypertensive encephalopathya']
Table 1 has SOC elements
The table:  1 1st Row:  0    System organ \nclass
1             Very common
2                  Common
3                    Rare
4               Very rare
5               Not known
Name: 0, dtype: object
Case C: Table has frequencies in the first row
Frequencies:  {1: 'Very common', 2: 'Common', 3: 'Rare', 4: 'Very rare', 5: 'Not known'}
AR list: ['Gastrointestinal ulcera']
AR list: ['Palmar']
AR list: ['Lethargy']
AR list: ['Weight decreased']
Table 2 has SOC elements
The

/var/folders/ks/61tstkzx4rj7v0my891xbvx40000gn/T/ipykernel_41646/4132718398.py:39: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if df[0].str.contains(regexSOC, regex=True).any():
/var/folders/ks/61tstkzx4rj7v0my891xbvx40000gn/T/ipykernel_41646/4132718398.py:81: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  elif df.iloc[0].astype(str).str.contains(freq_regex).any():
/var/folders/ks/61tstkzx4rj7v0my891xbvx40000gn/T/ipykernel_41646/4132718398.py:39: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if df[0].str.contains(regexSOC, regex=True).any():
/var/folders/ks/61tstkzx4rj7v0my891xbvx40000gn/T/ipykernel_41646/4132718398.py:81: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually g